In [1]:
import pandas as pd
import numpy as np

# Constants and EV specifications
year = 2016
hours_per_year = 8784
car_efficiency = 191 / 1000  # kWh per km
battery_capacity = 72  # kWh
battery_min = 10  # %
battery_max = 80  # %
min_battery_level = battery_min / 100 * battery_capacity  # kWh
max_battery_level = battery_max / 100 * battery_capacity  # kWh
max_charge_power = 11  # kW

# Load PV and BS data (adjust the path as needed)
pv_bs_profile_path = 'C:\\Users\\DianaEspinosaLozano\\anaconda3\\envs\\thesis\\GETTINGSTARTED\\df_pv_bs_240906.csv'
pv_bs_profile = pd.read_csv(pv_bs_profile_path)

# Results DataFrame setup
results = pd.DataFrame(index=pd.date_range(start=f'{year}-01-01', periods=hours_per_year, freq='h'), 
                       columns=['EV_battery', 'EV_at_home', 'PV_used', 'Grid_used', 'BS_used', 'Energy_used_total'])

def simulate_ev_charging(homeoffice=True):
    # Load the pre-saved car availability profile with distance traveled based on the homeoffice setting
    profile_name = f'C:\\Users\\DianaEspinosaLozano\\anaconda3\\envs\\thesis\\car_availability_homeoffice_{homeoffice}.csv'
    profile_data = pd.read_csv(profile_name, index_col=0)
    car_availability = profile_data['EV_at_home'].values
    distance_travelled = profile_data['distance_travelled'].values

    # Initial conditions
    ev_battery = max_battery_level
    bs_soc = pv_bs_profile.loc[0, 'battery_soc']  # Start with the initial battery storage state

    for hour in range(hours_per_year):
        pv_used_total = 0
        grid_used_total = 0
        bs_used_total = 0
        
        # Driving consumption logic: Deduct battery consumption based on precomputed distance
        trip_distance = distance_travelled[hour]
        if car_availability[hour] == 0 and trip_distance > 0:
            # Calculate required battery for the trip
            required_battery = trip_distance * car_efficiency
            if ev_battery - required_battery >= min_battery_level:
                ev_battery -= required_battery
            else:
                # If not enough battery for the full trip, reduce distance proportionally
                trip_distance = (ev_battery - min_battery_level) / car_efficiency
                ev_battery = min_battery_level  # Battery depletes to minimum level only

        # Charging logic (if car is at home)
        if car_availability[hour] == 1 and ev_battery < max_battery_level:
            # Check PV surplus and BS state
            pv_surplus = pv_bs_profile.loc[hour, 'pv_excess']
            bs_soc = pv_bs_profile.loc[hour, 'battery_soc']
            pv = pv_bs_profile.loc[hour, 'pv']
            
            # Charging prioritization
            charge_power = min(max_charge_power, max_battery_level - ev_battery)

            if pv_surplus >= 1.4:  # Charge entirely from PV surplus if sufficient
                charge_power = min(pv_surplus, charge_power)
                ev_battery += charge_power
                pv_used_total += charge_power
            
            elif 0.9 < pv_surplus < 1.4:  # Part PV, part grid charging
                grid_contribution = 1.4 - pv_surplus  # Grid supplement to reach 1.4 kW
                charge_power = min(1.4, charge_power)  # Cap charge at 1.4 kW total
                ev_battery += charge_power
                pv_used_total += pv_surplus
                grid_used_total += grid_contribution
            
            elif pv < 10 and bs_soc > 1.4:  # Use battery storage if PV is low
                bs_available_power = max(0, 10 - pv)
                charge_power = min(bs_available_power, charge_power)
                ev_battery += charge_power
                bs_used_total += charge_power
                bs_soc -= charge_power  # Update battery storage SOC
                
            elif ev_battery > min_battery_level and ev_battery <= 20:  # Charge from grid if low
                charge_power = min(1.4, charge_power)
                ev_battery += charge_power
                grid_used_total += charge_power

        # Calculate total energy used for charging the EV this hour
        energy_used_total = pv_used_total + grid_used_total + bs_used_total

        # Update results
        results.loc[results.index[hour], 'EV_battery'] = ev_battery
        results.loc[results.index[hour], 'EV_at_home'] = car_availability[hour]
        results.loc[results.index[hour], 'PV_used'] = pv_used_total
        results.loc[results.index[hour], 'Grid_used'] = grid_used_total
        results.loc[results.index[hour], 'BS_used'] = bs_used_total
        results.loc[results.index[hour], 'Energy_used_total'] = energy_used_total
        
    return results


#### Homeoffice = True

In [2]:
res = simulate_ev_charging(homeoffice=True)
res

,EV_battery,EV_at_home,PV_used,Grid_used,BS_used,Energy_used_total
2016-01-01 00:00:00,57.6,1.0,0,0,0,0
2016-01-01 01:00:00,57.6,1.0,0,0,0,0
2016-01-01 02:00:00,57.6,1.0,0,0,0,0
2016-01-01 03:00:00,57.6,1.0,0,0,0,0
2016-01-01 04:00:00,57.6,1.0,0,0,0,0
...,...,...,...,...,...,...
2016-12-31 19:00:00,57.6,1.0,0,0,0,0
2016-12-31 20:00:00,57.6,1.0,0,0,0,0
2016-12-31 21:00:00,57.6,1.0,0,0,0,0
2016-12-31 22:00:00,57.6,1.0,0,0,0,0


In [ ]:
# Calculate sums
grid = sum(res['Grid_used'])
pv = sum(res['PV_used'])
bs = sum(res['BS_used'])
energy = sum(res['Energy_used_total'])
home = sum(res['EV_at_home'])

# Print results
print('Grid: ', grid)
print('PV used: ', pv)
print('BS used: ', bs)
print('Energy used total: ', energy)
print('EV at home: ', home)
print('EV not at home: ', hours_per_year - home)

Grid:  7.72186326232209
PV used:  393.19590383828273
BS used:  1542.316232899395
Energy used total:  1943.2339999999997
EV at home:  7654.0
EV not at home:  1130.0


#### Homeoffice = False

In [5]:
res_f = simulate_ev_charging(homeoffice=False)

# Calculate sums
grid = sum(res_f['Grid_used'])
pv = sum(res_f['PV_used'])
bs = sum(res_f['BS_used'])
energy = sum(res_f['Energy_used_total'])
home = sum(res_f['EV_at_home'])

# Print results
print('Grid: ', grid)
print('PV used: ', pv)
print('BS used: ', bs)
print('Energy used total: ', energy)
print('EV at home: ', home)
print('EV not at home: ', hours_per_year - home)

Grid:  72.80940097832489
PV used:  509.8763216486595
BS used:  2782.3522773730147
Energy used total:  3365.037999999999
EV at home:  6217.0
EV not at home:  2567.0
